##parser 1

In [ ]:
reviews = rv.query_selector_all('[data-name="ReviewsItem"]')
                logging.info(f"Найдено отзывов: {len(reviews)} для «{project_name}»")

                for rev in reviews:
                    raw_head = rev.inner_text().split("\n", 1)[0]

                    info_div = rev.query_selector("div[class*='--info--']")
                    review_date = rating = ""
                    if info_div:
                        date_el = info_div.query_selector("span")
                        if date_el:
                            review_date = date_el.inner_text().strip()
                        rating = str(sum(
                            1 for li in info_div.query_selector_all("li")
                            if "-1" in (li.inner_html() or "")
                        )) or ""

                    if not review_date:
                        m = re.search(r"(\d{1,2} [А-Яа-я]+ 20\d{2})", raw_head)
                        review_date = m.group(1) if m else ""

                    username = re.sub(r"\d{1,2} [А-Яа-я]+ 20\d{2}", "", raw_head)
                    username = re.sub(r"[*★☆•·]", "", username).strip()

                    sum_el = rev.query_selector('[class*="--title--"]')
                    summary = sum_el.inner_text().strip() if sum_el else ""

                    adv = dis = ""
                    for part in rev.query_selector_all("div[data-name='CommentPartComponent']"):
                        lbl = part.query_selector("div[class*='--title']")
                        txt_el = part.query_selector("div[class*='--text']")
                        txt = txt_el.inner_text().strip() if txt_el else ""
                        L = lbl.inner_text().strip() if lbl else ""
                        if "Плюсы" in L:
                            adv = txt
                        elif "Минусы" in L:
                            dis = txt

                    if not summary and not adv and not dis:
                        txt_el = rev.query_selector(
                            "div[data-name='CommentPartComponent'] div[class*='--text']"
                        )
                        if txt_el:
                            summary = txt_el.inner_text().strip()

                    writer.writerow({
                        "developer": developer,
                        "username": username,
                        "review_date": review_date,
                        "rating": rating,
                        "project_name": project_name,
                        "summary": summary,
                        "advantages": adv,
                        "disadvantages": dis
                    })
                rv.close()
                time.sleep(random.uniform(1, 2))

        browser.close()
    csv_file.close()
    logging.info("=== Завершено: результаты в cian_reviews_optima.csv ===")
    print("Парсинг завершён. Результаты в cian_reviews_optima.csv")


if name == "__main__":
    t = threading.Thread(target=run_scraper)
    t.start()
    t.join()

## parser 2

In [ ]:
# читаем ценовые блоки
            async def read_price_blocks():
                b = await page.query_selector('[data-testid="valuation_summary_avgPrice"]')
                d = await page.query_selector('[data-testid="valuation_summary_avgPriceSqm"]')
                if not (b and d):
                    return (None, None, None, None)
                try:
                    text_b, text_d = await b.inner_text(), await d.inner_text()
                    if not text_b.strip():
                        return (None, None, None, None)
                except Exception:
                    return (None, None, None, None)

                def try_num(fn):
                    try:
                        return fn()
                    except Exception:
                        return None

avg_price = try_num(lambda: clean_number(only_digits(text_b)))
                avg_sqm   = try_num(lambda: clean_number(only_digits(text_b.split("₽")[-2])))
                delta_abs = try_num(lambda: clean_number(only_digits(text_d.split("₽")[0])))

                pct_span = await d.query_selector("span[class*='--price-diff--'] span")
                delta_pct = None
                if pct_span:
                    pct_text = await pct_span.inner_text()
                    delta_pct = try_num(lambda: int(clean_number(only_digits(pct_text))))

                return avg_price, avg_sqm, delta_abs, delta_pct

            avg_flat_price, avg_sqm_price, flat_price_change, pct_flat_price_change = await read_price_blocks()

            # подсчитываем количество наград
            cnt_awards = len(await page.query_selector_all(
                "div[class*='--slider--'] div[class*='--card--']"
            )) or None

            # цены по вкладкам «Количество комнат»
            room_chips = [
                ("Студия", "studio"),
                ("1",      "1_room"),
                ("2",      "2_room"),
                ("3",      "3_room"),
                ("4+",     "4_room"),
            ]
            per_room_data = {key: (None, None, None, None) for _, key in room_chips}

            if flat_container:
                for label_txt, key in room_chips:
                    chip = await flat_container.query_selector(f"label:has-text('{label_txt}')")
                    if not chip:
                        continue
                    await chip.click()
                    await asyncio.sleep(1.2)
                    per_room_data[key] = await read_price_blocks()

            row = {
                "developer_name"      : developer_name,
                "year_of_foundation"  : year_of_foundation,
                "cnt_houses_building" : cnt_houses_building,
                "cnt_houses_built"    : cnt_houses_built,
                "work_regions"        : work_regions,
                "cnt_work_regions"    : cnt_work_regions,
                "estate_class_list"   : estate_class_list,
                "cnt_estate_class"    : cnt_estate_class,
                "room_types"          : room_types,
                "cnt_room_types"      : cnt_room_types,
                "avg_flat_price"      : avg_flat_price,
                "avg_sqm_price"       : avg_sqm_price,
                "flat_price_change"   : flat_price_change,
                "%_flat_price_change" : pct_flat_price_change,
                "cnt_awards"          : cnt_awards,
            }

            for k in ("studio", "1_room", "2_room", "3_room", "4_room"):
                avg_p, avg_sqm, delta_abs, delta_pct = per_room_data[k]
                row[f"avg_{k}_price"]      = avg_p
                row[f"avg_{k}_sqm_price"]  = avg_sqm
                row[f"{k}_price_change"]   = delta_abs
                row[f"%_{k}_price_change"] = delta_pct

            writer.writerow(row)
            await asyncio.sleep(random.uniform(1, 2))
            await ctx.clear_cookies()

        await browser.close()
    csv_file.close()
    print("Парсинг завершён  →  zastroischiki_info.csv создан.")

if name == "__main__":
    asyncio.run(run_scraper())